In [3]:
import json
from bs4 import BeautifulSoup

The purpose of this notebook is to get all my beautifulSoup calls correctly formatted

In [6]:
path = '../chic_data/chic_export_1.json'
is_exists = []
check = []
features_collect = []

In [7]:
with open(path) as json_data:
    for line in json_data:
        entry = json.loads(line)

In [15]:
entry
fd = {}

In [16]:
def check_status(html):
    '''some entries either refer to urls with no photo or return an error
    this function will alert to pages that do not exist and flag pages that should
    rechecked'''
    soup = BeautifulSoup(html, "html.parser")
    output = ''
    text = str(soup.find())
    if text.startswith('<response [200]') or text.startswith('<response [404]'):
        output = 'check'
    elif soup.title.text == u"The page you are looking for doesn't exist | Chictopia":
        output = "DNE"
    else:
        output = "passed"
    return output

In [17]:
html = entry['html'] 

soup = BeautifulSoup(html, "html.parser")

In [18]:
title = soup.find("h1", { "class":"photo_title"}).text
title

u'freaky friday'

In [19]:
#I would like to be able to extract just the urls
sub_photos = soup.find("div", {"class":"subphoto_items"})
if sub_photos is not None:
    sub_photos = sub_photos.findChildren("img")
    sub_photos = [photo.get("src") for photo in sub_photos]
    str(sub_photos)
else:
    "n/a"

In [20]:
#here also src would be helpful
main_photo = soup.find("div", {"id" : "image_wrap"}).findChild("img").get("src")
main_photo


u'http://images0.chictopia.com/photos/fununderthesun/5013795741/5013795741_400.jpg'

In [21]:
votes = soup.find("div", {"id": "vote_text_100000"})
if votes is not None:
    votes = votes.text
    votes = int(votes.split()[0])
    fd['votes'] = int(votes)
else:
    fd['votes'] = 0

In [22]:
comments = soup.find("div", {"id": "comment_text_100000"})
if comments is not None:
    comments = comments.text
    comments = int(comments.split()[0])
    fd['comments'] = int(comments)
else:
    fd['comments'] = 0

In [23]:
faves = soup.find("div", {"id": "favorite_text_100000"})
if faves is not None:
    faves = faves.text
    faves = int(faves.split()[0])
    fd['favorites'] = int(faves)
else:
    fd['faves'] = 0

In [24]:
tags = soup.find("div", {"id": "tag_boxes"}).findChildren("a")
tags = [tag.text for tag in tags]
tags


[u'Casual', u'Cocktail', u'fall']

In [25]:
desc = soup.find("div", {"id": "photo_description"}).text
desc

u'\nscarf and belt vintage.\ndress topshop.\nsilver sling bag topshop. '

In [26]:
#this doesn't exist on this page. I probably need a contingency for any time an element doesn't exist

links = soup.find("div",{"class": "garmentLinks"})
links

In [27]:
style_council = soup.find("div", {"class":"help"}).findChild("img").get("alt")
style_council

u'Style_council'

In [38]:
#need contingency for when people have 0 followers like here
followers = soup.find("div", {"id":"follow"}).text
if followers != '\n\n':
    print followers
else:
    print "no followers"
    

no followers


In [37]:
soup.find("div", {"id":"follow"}).text

u'\n\n'

In [ ]:
username = soup.find("div",{ "id":"name_div"}).findChild("a").text
username

In [ ]:
location = soup.find("div", {"id":"loc_div"}).findChild("a").text
location

In [ ]:
chic_points = soup.find("div", {"itemprop": "author"}).findChildren("div", {"class":"px10"})[2].text
chic_points = chic_points.split()[0]
chic_points

In [ ]:
date = soup.find("meta", {"itemprop":"dateCreated"}).get("content")
date

In [ ]:
#create a features dictionary
fd = {}

In [ ]:
labels = [username, title, date, location, desc, chic_points, followers, votes, comments, faves, \
              style_council, links, tags, main_photo, sub_photos]

